<a href="https://colab.research.google.com/github/jayktee/metis/blob/master/nlp/project/BERTopictweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic
#!pip install bertopic[visualization]
import pandas as pd
from bertopic import BERTopic

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [82]:
tweets = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/data/tweets_lem_stem2.csv')
tweets = tweets[tweets.tweet_lemmed.notna()].reset_index(drop=True)
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format='%Y-%m-%d', errors='coerce')
tweets = tweets[tweets['created_at'].notna()].reset_index(drop=True)
tweets = tweets.sort_values(by='created_at', ascending=True, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,1,7,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.



In [84]:
len(tweets)

95244

In [85]:
timestamps = tweets['created_at']
tweets = tweets['tweet_lemmed']

In [86]:
len(tweets)

95244

In [87]:
tweets.sample(5)

51833    a parent forbid covid shot defiant teenager se...
50106    houston methodist staffer lawsuit challenging ...
31752    getting ready to go get my   dose of moderna c...
19166    my work week is almost over i don’t feel any m...
57677    people that are surprised that they got covid ...
Name: tweet_lemmed, dtype: object

## Data cleaning and training model

### Testing Vectorizer

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['like', 'wa', 'ha', 'um', 'amp', 'le', 'im', 'ie'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [88]:
tv = TfidfVectorizer(stop_words=stopwords, max_df=0.90, min_df=0.0001)
doc_term_tv = tv.fit_transform(tweets)

In [89]:
doc_term_tv.shape

(95244, 8153)

In [ ]:
tv.vocabulary_

In [91]:
topic_model = BERTopic(top_n_words=10, min_topic_size=20, verbose=True, vectorizer_model=tv)
topics, probs = topic_model.fit_transform(tweets)

Batches:   0%|          | 0/2977 [00:00<?, ?it/s]

2022-01-25 02:03:00,932 - BERTopic - Transformed documents to Embeddings
2022-01-25 02:05:45,196 - BERTopic - Reduced dimensionality with UMAP
2022-01-25 02:05:57,725 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [92]:
topic_model.save("/content/gdrive/MyDrive/Colab Notebooks/data/bertopic_model_4")

In [93]:
import pickle
with open("/content/gdrive/MyDrive/Colab Notebooks/data/topics.pkl", 'wb') as f:
  pickle.dump(topics, f)

with open("/content/gdrive/MyDrive/Colab Notebooks/data/probs.pkl", 'wb') as f:
  pickle.dump(probs, f)

In [ ]:
# Further reduce topics
# new_topics, new_probs = topic_model.reduce_topics(docs, topics, nr_topics=30)

## Loading model

In [94]:
topic_model = BERTopic.load("/content/gdrive/MyDrive/Colab Notebooks/data/bertopic_model_4")

In [95]:
topic_model.visualize_barchart()

In [96]:
topic_model.visualize_topics()

In [97]:
timestamps

0       2021-01-01 00:04:14+00:00
1       2021-01-01 00:05:30+00:00
2       2021-01-01 00:07:51+00:00
3       2021-01-01 00:11:27+00:00
4       2021-01-01 00:14:05+00:00
                   ...           
95239   2021-12-30 23:44:44+00:00
95240   2021-12-30 23:45:34+00:00
95241   2021-12-30 23:47:03+00:00
95242   2021-12-30 23:54:15+00:00
95243   2021-12-30 23:55:43+00:00
Name: created_at, Length: 95244, dtype: datetime64[ns, UTC]

In [98]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamps, nr_bins=6, global_tuning=False)

6it [00:12,  2.11s/it]


In [99]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

## Use Tfidf with BERTopic

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['like', 'wa', 'ha', 'um', 'amp', 'le', 'im', 'ie'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
tv = TfidfVectorizer(stop_words=stopwords, max_df=0.1, min_df=0.001)

In [43]:
topic_model_tv = BERTopic(top_n_words=10, min_topic_size=20, verbose=True, vectorizer_model=tv)
topics_tv, probs_tv = topic_model_tv.fit_transform(tweets)

Batches:   0%|          | 0/1980 [00:00<?, ?it/s]

2022-01-20 23:09:22,560 - BERTopic - Transformed documents to Embeddings
2022-01-20 23:11:11,702 - BERTopic - Reduced dimensionality with UMAP
2022-01-20 23:11:20,638 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [44]:
topic_model_tv.save("bertopic_model_tv_2")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [45]:
topic_model_tv.save("/content/gdrive/MyDrive/Colab Notebooks/data/bertopic_model_tv_2")

In [46]:
topics_over_time_tv = topic_model_tv.topics_over_time(tweets, topics_tv, timestamps, nr_bins=4, global_tuning=False)

4it [00:07,  1.91s/it]


In [7]:

topic_model_tv = bertopic.load("/content/gdrive/MyDrive/Colab Notebooks/data/bertopic_model_tv_2")

NameError: ignored

In [1]:
topic_model_tv.visualize_topics()

NameError: ignored

In [49]:
topic_model_tv.get_topic_info()

,Topic,Count,Name
0,-1,28641,-1_triangularflag_district_looting_pain
1,0,1365,0_distancing_indoors_wore_indoor
2,1,1088,1_scheduling_hosting_armory_booking
3,2,1041,2_kids_pediatrician_pediatric_childrens
4,3,916,3_reduces_severity_lessens_lessen
...,...,...,...
302,304,21,304_nba_quiet_lebron_goodwin
300,303,21,303_dallas_houston_courtapproved_takeyomeds
307,306,20,306_osumedicinecn_amina_bared_osumedicine
308,307,20,307_justaskin_onondaga_contraindication_spoiler


In [48]:
topic_model_tv.visualize_topics_over_time(topics_over_time_tv, top_n_topics=10)